***Installation***

In [ ]:
!pip install -q openai-whisper
!sudo apt-get install -y ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.5 MB/

**Taking Voice Command from User And Converting into Raw Text**

In [ ]:
# STEP 2: Transcribe using Whisper
import whisper

# Load the Whisper model
whisper_model = whisper.load_model("base")

# Transcribe audio file
result = whisper_model.transcribe("/content/QuadCommandOne.m4a")
raw_text = result["text"]

print("Raw Transcription:", raw_text)


Raw Transcription:  Drone, take off and after that move forward and then hold there.


# Refining Raw text using flan-t5 model

In [ ]:
!pip install -q transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load Flan-T5
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define the correction prompt
from transformers import pipeline
corrector = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

instruction = f"the following text has come from speech to text coversion where speech is related to drone,correct the raw text:\n{raw_text}"

# Get response
result = corrector(instruction, max_length=100)
cleaned_text = result[0]['generated_text']

print("✅ Cleaned Text:", result[0]['generated_text'])


Device set to use cuda:0


✅ Cleaned Text: Drone, take off and after that move forward and then hold there.


# Setting up openai api key

In [ ]:
!pip install openai


In [ ]:
from openai import OpenAI
import os

client = OpenAI(api_key="My API KEY")  # Replace with your real API key


# Generating code from text in accordance with necessary prompts

In [ ]:
def generate_mavsdk_code(command_text):
    prompt = f"""
You are an expert in MAVSDK Python library for drone programming . Convert the following natural language instruction into valid MAVSDK Python code.

Instruction:
"{command_text}"

Your response should be a complete, working script using MAVSDK, using async/await, drone connection, arming, flight steps, and return to home if needed. Output only code.Remember to execute instuctions only.
In order to move drone in forward,backward,left,right ,up,down direction use this :
   print("-- Setting initial setpoint")
    await drone.offboard.set_velocity_body(
        VelocityBodyYawspeed(0.0, 0.0, 0.0, 0.0))

    print("-- drone attitude control")
    await drone.offboard.set_velocity_body(
        VelocityBodyYawspeed(0.0, 0.0, 0.0, 0.0))
    await asyncio.sleep(5)

in this the parameters are in order (Vx,Vy,Vz,yaw velocity).

in order to execute a circle use :
"    print("-- Fly a circle")
    await drone.offboard.set_velocity_body(
        VelocityBodyYawspeed(5.0, 0.0, 0.0, 30.0))
    await asyncio.sleep(5)
"
"""

    response = client.chat.completions.create(
        model="gpt-4o",  # 🧠 Top-tier model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1
    )

    return response.choices[0].message.content


In [ ]:
final_code = generate_mavsdk_code(cleaned_text)
print("🚁 Generated MAVSDK Code:\n")
print(final_code)


🚁 Generated MAVSDK Code:

```python
import asyncio
from mavsdk import System
from mavsdk.offboard import VelocityBodyYawspeed

async def run():
    drone = System()
    await drone.connect(system_address="udp://:14540")

    print("Waiting for drone to connect...")
    async for state in drone.core.connection_state():
        if state.is_connected:
            print(f"Drone discovered!")
            break

    print("-- Arming")
    await drone.action.arm()

    print("-- Taking off")
    await drone.action.takeoff()
    await asyncio.sleep(5)

    print("-- Setting initial setpoint")
    await drone.offboard.set_velocity_body(
        VelocityBodyYawspeed(0.0, 0.0, 0.0, 0.0))

    print("-- Starting offboard")
    try:
        await drone.offboard.start()
    except Exception as e:
        print(f"Failed to start offboard mode: {e}")
        return

    print("-- Moving forward")
    await drone.offboard.set_velocity_body(
        VelocityBodyYawspeed(5.0, 0.0, 0.0, 0.0))
    await as

# The above code is tested on quadcopter in simulation
